In [1]:
import time

from pycram import neem_loader as nl
from pycram.datastructures.enums import ObjectType, WorldMode
from pycram.world_concepts.world_object import Object
from pycram.worlds.bullet_world import BulletWorld
from sqlalchemy import create_engine

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Failed to import Giskard messages
Could not import RoboKudo messages, RoboKudo interface could not be initialized
pybullet build time: Nov 28 2023 23:51:11


In [2]:
# Connection to MariaDB NEEM database.
sql_engine = create_engine("mysql+pymysql://neem@neem-2.informatik.uni-bremen.de/neems")

In [3]:
all_neems_df = nl.get_dataframe_from_sql_query_file('../resources/get_pouring_neems.sql', sql_engine)

In [4]:
# neem_ids = nl.get_neem_ids(all_neems_df)
# neem_df = nl.filter_by_neem_id(all_neems_df, neem_ids[0])
neem_df = nl.filter_by_participant_type(all_neems_df, 'soma:DesignedContainer')
# neem_df = nl.normalize_time(neem_df)
participants = nl.get_participants_per_neem(neem_df)
print(participants)
environment = nl.get_environment(neem_df)
environment_path = 'apartment.urdf' if environment[0] == 'Kitchen' else environment[0]
participants_path = {}
for n, p in participants:
    if 'cup' in p.lower():
        participants_path[(n,p)] = 'jeroen_cup.stl'
    elif 'bowl' in p.lower() or 'pot' in p.lower():
        participants_path[(n,p)] = 'bowl.stl'
    elif 'pitcher' in p.lower():
        participants_path[(n,p)] = 'Static_MilkPitcher.stl'
    elif 'milk' in p.lower():
        participants_path[(n,p)] = 'milk.stl'
    elif 'bottle' in p.lower():
        participants_path[(n,p)] = 'Static_CokeBottle.stl'
    elif 'cereal' in p.lower():
        participants_path[(n,p)] = 'cereal.stl'
    else:
        participants_path[(n,p)] = 'generic_object.stl'

[('63d93736bc1f60335b4418d7', 'iai-apartment:SM_BigBowl_1'), ('63d93736bc1f60335b4418d7', 'iai-apartment:jeroen_cup_1'), ('6411e70a3cf55ecbffe2e51a', 'iai-apartment:SM_BigBowl_1'), ('6411e70a3cf55ecbffe2e51a', 'iai-apartment:SM_Pot_1'), ('6413244caf7532c49c6d80b0', 'iai-apartment:SM_Pitcher_1'), ('6413244caf7532c49c6d80b0', 'iai-apartment:SM_BigBowl_1'), ('6413244caf7532c49c6d80b0', 'iai-apartment:SM_MilkPitcher_1'), ('6413244caf7532c49c6d80b0', 'iai-apartment:SM_CokeBottle_1')]


In [5]:
world = BulletWorld(mode=WorldMode.GUI)
kitchen = Object(environment[0], ObjectType.ENVIRONMENT, environment_path)
neem_ids = nl.get_neem_ids(neem_df)
participants_objects = {}
for n, p in participants:
    if 'bowl' in p.lower() or 'pot' in p.lower():
        obj_type = ObjectType.BOWL
    elif 'milk' in p.lower():
        obj_type = ObjectType.MILK
    elif 'cup' in p.lower():
        obj_type = ObjectType.JEROEN_CUP
    else:
        obj_type = ObjectType.GENERIC_OBJECT
    participants_objects[(n,p)] = Object(p, obj_type, participants_path[(n,p)])

Unknown tag "material" in /robot[@name='apartment']/link[@name='coffe_machine']/collision[1]
Unknown tag "rgba_color" in /robot[@name='iai-apartment:SM_BigBowl_1_object']/link[@name='iai-apartment:SM_BigBowl_1_main']/visual[1]/material[@name='white']
Unknown tag "material" in /robot[@name='apartment']/link[@name='coffe_machine']/collision[1]
Unknown tag "rgba_color" in /robot[@name='iai-apartment:jeroen_cup_1_object']/link[@name='iai-apartment:jeroen_cup_1_main']/visual[1]/material[@name='white']
Unknown tag "rgba_color" in /robot[@name='iai-apartment:SM_BigBowl_1_object']/link[@name='iai-apartment:SM_BigBowl_1_main']/visual[1]/material[@name='white']
Unknown tag "rgba_color" in /robot[@name='iai-apartment:SM_BigBowl_1_object']/link[@name='iai-apartment:SM_BigBowl_1_main']/visual[1]/material[@name='white']
Unknown tag "rgba_color" in /robot[@name='iai-apartment:SM_BigBowl_1_object']/link[@name='iai-apartment:SM_BigBowl_1_main']/visual[1]/material[@name='white']
Unknown tag "rgba_color"

In [6]:
poses = nl.get_poses(neem_df)
times = nl.get_stamp(neem_df)
participant_names = nl.get_participants(neem_df, unique=False)
neem_ids = nl.get_neem_ids(neem_df, unique=False)
prev_time = 0
for n, p, pose, current_time in zip(neem_ids, participant_names, poses, times):
    if prev_time > 0:
        wait_time = current_time - prev_time
        if wait_time > 1:
            wait_time = 1
        time.sleep(wait_time)
    prev_time = current_time
    participants_objects[(n, p)].set_pose(pose)

In [7]:
world.exit()

Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/bassioun/cram_ws/src/pycram/src/pycram/worlds/bullet_world.py", line 381, in run
    keys = p.getKeyboardEvents()
pybullet.error: Not connected to physics server.


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 4070/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 535.161.07
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 535.161.07
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 4070/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation
